# Using Basic Marshmallow without SQL Alchemy Integration

In [ ]:
from os import environ
import urllib
#import json

from sqlalchemy import create_engine, inspect
from sqlalchemy import Table, Column, ForeignKey, Integer, String, Float, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import sessionmaker, selectinload
from sqlalchemy.sql import union, select, and_, or_, not_, text
from sqlalchemy.sql import bindparam
from sqlalchemy.sql.functions import coalesce

from marshmallow import Schema, fields, validate, EXCLUDE, pprint

# Setup

In [ ]:
driver = environ.get('SQL_DRIVER', '{ODBC Driver 17 for SQL Server}')
host = environ.get('SQL_HOST', 'sql-fabulous')
db = environ.get('SQL_DB', 'ScratchDB')
user = environ.get('SQL_USER', 'sa')
pw = environ.get('SQL_PASSWORD', 'HelloWorld1')

con_str = f'DRIVER={driver};SERVER={host};DATABASE={db};UID={user};PWD={pw}'

params = urllib.parse.quote_plus(con_str)  

# 'echo' emits generated sql
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", echo=True)

# Define ORM Schema

In [ ]:
Base = declarative_base()

class Survey(Base):
    __tablename__ = 'DirectionalSurvey'

    ID = Column(Integer, autoincrement=False, primary_key=True)
    API = Column(String(32), nullable=True)
    WKID = Column(String(32), nullable=True)
    FIPS = Column(String(4), nullable=True)
    STATUS_CODE = Column(String(1), nullable=False)

    def __repr__(self):
        return f"Survey(ID={self.ID}, API={self.API}, WKID={self.WKID}, Points={self.stations})"

class SurveyReport(Base):
    __tablename__ = 'SurveyReport'

    ID = Column(Integer, autoincrement=False, primary_key=True)
    DirectionalSurveyId = Column(Integer, ForeignKey('DirectionalSurvey.ID'), nullable=False)
    Azimuth = Column(Float, nullable=True)
    MD = Column(Float, nullable=True)
    Inclination = Column(Float, nullable=True)
    STATUS_CODE = Column(String(1), nullable=False)
    survey = relationship(Survey, backref=backref('stations', uselist=True))

    def __repr__(self):
        return f"Report(ID={self.ID}, FK={self.DirectionalSurveyId}, STATUS={self.STATUS_CODE})"

# Set mapped tables to local vars so that full SQL metadata is available.
surveys = Survey.__table__
points = SurveyReport.__table__

# Define Marshmallow Schema

In [ ]:
class SurveySchema(Schema):
    # Ignore unknown fields
    class Meta:
        unknown = EXCLUDE
        ordered = True
    
    ID = fields.Int(required=True, data_key="SurveyId")
    API = fields.Str(validate=validate.Length(max=32))
    WKID = fields.Str(validate=validate.Length(max=32))
    FIPS = fields.Str(validate=validate.Length(max=4))
    STATUS_CODE = fields.Str(validate=validate.Length(1), required=True)
    
    stations = fields.Nested("SurveyReportSchema", many=True
                             , exclude=("DirectionalSurveyId", "survey"))

class SurveyReportSchema(Schema):
    class Meta:
        unknown = EXCLUDE
        ordered = True

    ID = fields.Int(required=True, data_key="SurveyReportId")
    DirectionalSurveyId = fields.Int(required=True)
    Azimuth = fields.Float()
    MD = fields.Float()
    Inclination = fields.Float()
    STATUS_CODE = fields.Str(validate=validate.Length(1), required=True)
    
    survey = fields.Nested(SurveySchema, many=False, exclude=("stations",))
    
survey_serdes = SurveySchema()  
report_serdes = SurveyReportSchema()

# Bind Engine to Schema and Create Session, Connection

In [ ]:
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()
conn = engine.connect()

# Select

In [ ]:
my_surveys = session.query(Survey) \
                .options(selectinload(Survey.stations))\
                .filter(Survey.STATUS_CODE.in_(['N', 'C']))\
                .filter(Survey.ID < 2)\
                .filter(Survey.stations.any())\
                .all()

try:
    for s in my_surveys:
        print(f"S: {type(s)}, {s}")
        s_json = survey_serdes.dump(s)
        pprint(s_json, indent=2)
        
        for sr in s.stations:
            print(f"SR: {type(sr)}, {sr}")
            sr_json = report_serdes.dump(sr)
            pprint(sr_json, indent=2)
        
except Exception as e:
    print(f"{e}")

In [ ]:
my_surveys = session.query(Survey)\
                .join(SurveyReport)\
                .filter(Survey.STATUS_CODE.in_(['N', 'C']))\
                .filter(Survey.WKID.like('WKID2%'))\
                .filter(SurveyReport.Azimuth >= 7)\
                .all()

try:
    for s in my_surveys:
        print(f"S: {type(s)}, {s}")
        s_json = survey_serdes.dump(s)
        pprint(s_json, indent=2)
        
        for sr in s.stations:
            print(f"SR: {type(sr)}, {sr}")
            sr_json = report_serdes.dump(sr)
            pprint(sr_json, indent=2)
        
except Exception as e:
    print(f"{e}")